In [1]:
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install jieba
!pip install nltk
!pip install transformers
!pip install sentencepiece
!pip install hanlp
!pip install stanza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2023-04-28 09:27:48.273427: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-28 09:27:49.388331: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 79.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/p

In [2]:
import spacy
from jieba import posseg
from nltk.corpus import wordnet 
import jieba
import nltk
import stanza
import hanlp
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import re
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt", src_lang="zh_CN")
nltk.download('wordnet')
nlp_en = spacy.load("en_core_web_sm")
dep_zh = hanlp.load(hanlp.pretrained.dep.CTB9_DEP_ELECTRA_SMALL)
stanza.download('zh')
nlp_zh = stanza.Pipeline('zh')


[nltk_data] Downloading package wordnet to /root/nltk_data...
100%  47.2 MiB   7.2 MiB/s ETA:  0 s [=========================================]
Decompressing /root/.hanlp/dep/ctb9_dep_electra_small_20220216_100306.zip to /root/.hanlp/dep
100%  26.7 KiB  26.7 KiB/s ETA:  0 s [=========================================]
Decompressing /root/.hanlp/utils/char_table_20210602_202632.json.zip to /root/.hanlp/utils


Building model ...Downloading https://file.hankcs.com/hanlp/transformers/electra_zh_small_20210706_125427.zip to /root/.hanlp/transformers/electra_zh_small_20210706_125427.zip
100%  41.2 KiB  41.2 KiB/s ETA:  0 s [=========================================]
Decompressing /root/.hanlp/transformers/electra_zh_small_20210706_125427.zip to /root/.hanlp/transformers


INFO:stanza:"zh" is an alias for "zh-hans"
INFO:stanza:Downloading default packages for language: zh-hans (Simplified_Chinese) ...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:"zh" is an alias for "zh-hans"
INFO:stanza:Loading these models for language: zh-hans (Simplified_Chinese):
| Processor    | Package   |
----------------------------
| tokenize     | gsdsimp   |
| pos          | gsdsimp   |
| lemma        | gsdsimp   |
| constituency | ctb       |
| depparse     | gsdsimp   |
| sentiment    | ren       |
| ner          | ontonotes |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: constituency
INFO:stanza:Loading: depparse
INFO:stanza:Loading: sentiment
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
class Modulation:

    def __init__(self,path,save_path):
        with open(path) as f:
          text = f.readlines()
        self.en_li = []
        self.zh_li = []
        for i, sent in enumerate(text):
            if i%2 == 0:
                self.en_li.append(sent.strip())
            elif i%2 == 1:
                self.zh_li.append(sent.strip())
        self.save_path = save_path


    def negative(self): # 有not没有'无，不', vice versa
        
        e = 1
        for en,zh in zip(self.en_li, self.zh_li):
            en_tok_li = [en_tok.text for en_tok in nlp_en(en)]
            if 'not' in en_tok_li and 'or not' not in en and '不' not in zh and '没' not in zh and '未' not in zh and '非' not in zh and '无' not in zh:
                if en_tok_li.index('not') !=0 and en_tok_li.index('not') <len(en_tok_li)-3:
                    before_tok = en_tok_li[en_tok_li.index('not')-1]
                    after_tok = en_tok_li[en_tok_li.index('not')+1]
                    str_toks = before_tok+' not '+after_tok+':'
                    print(e)
                    e+=1
                    print(en)
                    print(zh)
                    print(str_toks)
                    print()
                    with open(save_path,'a+') as w:
                        w.write(en+'\n'+zh+'\n'+str_toks+'\n')
                



    def sub_obj(self):  # 包括passive voice to active voice   # 把字句，被字句
        e = 1
        r = 1
        regex1 = ' (is|are|was|were|be|been)(.{2,8}ed) '
        # if ',' not in en and '，' not in zh and ',' not in zh:
        for en,zh in zip(self.en_li, self.zh_li):
            
            passive_verb = re.findall(regex1,en)
            
            
            if passive_verb != [] and '把' in zh:
                # print(e)
                e += 1
                # print(en)
                # print(zh)
                # str_verb = passive_verb[0].strip()+':'
                # print(str_verb)
                # with open(save_path,'a+') as t:
                #     t.write(en+'\n'+zh+'\n'+str_verb+'\n')

            elif passive_verb != [] and '把' not in zh and len(passive_verb)==1:
                # print(en)
                # print(passive_verb)
                verb = ' '.join(passive_verb[0]).strip().split(" ")[-1]
                # print(verb)
                print(r)
                r += 1
                tokenized_verb_li =  [v.text for v in nlp_en(verb)]
                if len(tokenized_verb_li)==1:
                    tokenized_verb =  [v.text for v in nlp_en(verb)][0]
                    en_token = tokenizer(tokenized_verb, return_tensors="pt")
                    generated_tokens = model.generate(**en_token, forced_bos_token_id=tokenizer.lang_code_to_id["zh_CN"])
                    zh_text_trans = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
                    if zh_text_trans in zh and zh_text_trans != '' and zh_text_trans != ' ':
                        en_noun_li = [tok.text for tok in nlp_en(en.split(tokenized_verb)[0]) if tok.pos_ =='NOUN']
                        en2zh_noun_li = []
                        for en_noun in en_noun_li:
                            en_tokenn = tokenizer(en_noun, return_tensors="pt")
                            generated_token = model.generate(**en_tokenn, forced_bos_token_id=tokenizer.lang_code_to_id["zh_CN"])
                            zh_text = tokenizer.batch_decode(generated_token, skip_special_tokens=True)[0]
                            en2zh_noun_li.append(zh_text)
                            if zh.split(zh_text_trans)[0] != '' and en2zh_noun_li != []:
                                if len(list(set(jieba.cut(zh.split(zh_text_trans)[1])).intersection(set(en2zh_noun_li))))>0:
                                    print(en)
                                    print(zh)
                                    print(''.join(passive_verb[0]).strip())
                                    print("----------")
                                    with open(save_path,'a+') as y:
                                        y.write(en+'\n'+zh+'\n'+''.join(passive_verb[0]).strip()+':'+zh_text_trans+'\n')
                        





            
spath = '/content/gdrive/MyDrive/Colab Notebooks/literal translationese/dataset/Bilingual/Education/Bi-Education.txt'
save_path = '/content/gdrive/MyDrive/Colab Notebooks/literal translationese/dataset/feature67_mod_mod+transpo/mod_sub_obj_passive2active.txt'             
        
ptcl = Modulation(spath, save_path)

ptcl.sub_obj()


1


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (200) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Streaming output truncated to the last 5000 lines.
26088
26089
26090
26091
26092
26093
26094
26095
26096
26097
26098
26099
26100
26101
26102
26103
26104
26105
26106
26107
26108
26109
26110
26111
26112
26113
26114
26115
26116
26117
26118
26119
26120
26121
26122
26123
26124
26125
26126
26127
26128
26129
26130
26131
26132
26133
26134
26135
26136
26137
26138
26139
26140
26141
26142
26143
26144
26145
The ISOObp promoter of PNZIP gene was cloned by adaptor PCR method.
因此，我们利用接头PCR技术克隆了PNZIP基因的启动子。
was cloned
----------
26146
26147
26148
26149
26150
26151
26152
26153
26154
26155
26156
26157
26158
26159
26160
26161
26162
26163
26164
26165
26166
26167
26168
26169
26170
26171
26172
26173
26174
26175
26176
26177
26178
26179
Oil is traded in dollars, so a less valuable dollar buys less oil.
石油用美元交易，所以贬值的美元购买更少的石油。
is traded
----------
26180
26181
26182
26183
26184
26185
26186
26187
26188
26189
26190
26191
26192
26193
26194
26195
26196
26197
26198
26199
26200
26201
26202
The viscosity-composition c